# World Bank Data Project

### Data gathering.

Data are downloaded from the web and sorted in this manner

1. Data are first imported using bulk download options, by choosing only those that are updated at least last year 2017. This is done by looking at the 'Last Revision Date" column, anything that is either 'Current' or at has year '2017'. The import is done by the script 'import_data.py', which on top of downloading the data, they are extracted from zip files (if they are zip files) and stored in folders named after their names in 'Name' column. If they are not zip files, they are stored in either folders named 'csv_files' or 'excel_files'.

2. Zip files that are downloaded contains more than one file within, usually legends or footnotes, since I am only interested in data files, 'extract_data_files.py' is ran to check for data files and move them into the csv_files folder so all the data is collected in one folder.

3. The collective data files are sorted by their topic (moved into folders named after the topics) using 'sort_files_by_topic.py' which is done by checking the catalog for the the respective topics the data are related to. 

4. 'wrap_files_by_topic.py' is used to compile separate data files into one csv file for every topics covered. The csv files are structured as follow. *Note: not all file are csv files so they have to remove manually before wrapping. *

In [12]:
import pandas as pd
df = pd.read_csv('Economy.csv', index_col=0)
df.head()

,"GDP per capita, PPP (constant 2011 international $)",Net foreign assets (current LCU),"PPG, bonds (AMT, current US$)","Net bilateral aid flows from DAC donors, New Zealand (current US$)","Labor force participation rate, male (% of male population ages 15+) (national estimate)","Condom use at last high-risk sex, adult male (% ages 15-49)","Specialist surgical workforce (per 100,000 population)",Proportion of population spending more than 10% of household consumption or income on out-of-pocket health care expenditure (%),"Death rate, crude (per 1,000 people)","Communications, computer, etc. (% of service exports, BoP)",...,Insurance and financial services (% of commercial service imports),AIDS estimated deaths (UNAIDS estimates),Time spent dealing with the requirements of government regulations (% of senior management time),"PPG, bonds (DOD, current US$)","School enrollment, secondary, male (% net)",Coverage of unemployment benefits and ALMP in poorest quintile (% of population),"Health expenditure, total (% of GDP)",Imports of goods and services (% of GDP),Renewable internal freshwater resources per capita (cubic meters),Ratio of female to male labor force participation rate (%) (modeled ILO estimate)
Coverage_Range,"['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965...","['1990', '1991', '1992', '1993', '1994', '1995...","['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965...",...,"['1960', '1961', '1962', '1963', '1964', '1965...","['1990', '1991', '1992', '1993', '1994', '1995...","['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965...","['1960', '1961', '1962', '1963', '1964', '1965..."
Canada,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[127609919.69556999, 198946875.566513, -157815...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, 78.0999984741211, nan, nan, nan, nan, na...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[7.8, 7.7, 7.7, 7.8, 7.5, 7.6, 7.5, 7.4, 7.4, ...","[42.8003981847545, 23.632261743264397, 22.0022...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[18.135559267776, 18.051766928571602, 17.79289...","[nan, nan, 153110.56194262402, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
EMDE ECA South Caucasus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OECD members,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[10.3139925445923, 10.126629911490301, 10.2797...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,"[nan, nan, nan, nan, n

## Data Cleaning.
### For Visualisation

*full details can be found in 'Visualization_data_cleaning' notebook*

These functions are used to unwrap the dataframe for each indicators made earlier. (columns = countries, index = time series).

In [1]:
def unwrap(indicator_name, compiled_df):
    df_index = compiled_df.at['Coverage_Range', indicator_name]
    df_index = string_to_list(df_index)
    df_index = [ind for ind in df_index if len(ind)==4]
    df_index = pd.to_datetime(df_index)
    countries = compiled_df.index
    countries = countries.drop('Coverage_Range')
    data = {}
    for country in countries:
        annual_data = compiled_df.at[country,indicator_name]
        
        if not pd.isnull(annual_data):
            time_series = string_to_list(annual_data)
            if len(time_series)==len(df_index):            
            #print country
                data[country]=time_series
            #print len(time_series), country, len(df_index)
        else:
            data[country]=np.nan
    unwrapping = pd.DataFrame(index=df_index, data=data, dtype=np.dtype(float) )
    return unwrapping
def string_to_list(string):
    list_ = string[1:-1]
    list_ = list_.split(',')
    list_ = [t.replace("'", '').strip() for t in list_]
    return list_
def data_to_list(string):
    list_ = string[1:-1]
    list_ = list_.split(',')
    list_ = [t.replace("'", '').strip() for t in list_]
    list_ = [float(t) for t in list_ if t != 'nan']
    list_ = [np.nan for t in list_ if t == 'nan']
    
    return list_

This function is used to scrape country names. https://data.worldbank.org/country as source

In [2]:
def get_country_names_for_region(url):
    page = requests.get(url)
    soup4 = BeautifulSoup(page.text,'lxml')
    region = soup4.find_all('li', attrs={'class':'label'})
    names = [attr.text for attr in region]
    return names

This function is used to scrape for indicators for topics. https://data.worldbank.org/indicator as source

In [3]:
def get_topic_indicator_names(url,id):
    page = requests.get(url)
    soup4 = BeautifulSoup(page.text,'lxml')
    chart = soup4.find_all('ul',attrs={'data-reactid':'{}'.format(id)})
    s_list = chart[0].find_all('li')
    names = [attr.text for attr in s_list]
    return names

### For ML Models

*Full details can be found in the 'Model_data_cleaning_getting_indicators' notebook.*

This function is used to scrape for indicators for topics used in ML. https://data.worldbank.org/indicator?tab=all as source

In [4]:
def get_topic_indicator_names2(bsoup, react_id):
    chart = bsoup.find_all('ul',attrs={'data-reactid':'{}'.format(react_id)})
    s_list = chart[0].find_all('li')
    indicators = [attr.text for attr in s_list]
    return indicators

This function used clean dataframes after isolating the right dataframes.

In [5]:
def make_clean_df(clean_this_df):
    temp_dict = {}
    for col in clean_this_df.columns:
        ind_df = unwrap(col,clean_this_df)
        temp_dict[col]=ind_df.mean(axis=1)
        
    temp_index = temp_dict[clean_this_df.columns[0]].index
    cleaned_df = pd.DataFrame(data=temp_dict, index=temp_index)
    cleaned_df.dropna(axis=0, how='all', inplace=True)
    cleaned_df.dropna(axis=1, how='all', inplace=True)
    return cleaned_df

These functions are used to unwrap the wrapped dataframes by countries (columns = indicators, index = time series), then stacked together horizontally forming more samples for ML training.

In [ ]:
def string_to_list(string):
    list_ = string[1:-1]
    list_ = list_.split(',')
    list_ = [t.replace("'", '').strip() for t in list_]
    return list_

def country_unwrap_no_index(country, compiled_df):
    df_index = compiled_df.loc['Coverage_Range'].unique()[0]
    df_index = string_to_list(df_index)
    country_df = compiled_df.loc[country]
    data = {}
    for indicator, values in country_df.iteritems():
        if not pd.isnull(values):            
            time_series = string_to_list(values)
            if len(time_series)==len(df_index):
                data[indicator]=time_series
        else:
            data[indicator]=[np.nan]*len(df_index)
    unwrapping = pd.DataFrame(data=data, dtype=np.dtype(float) )
    unwrapping.interpolate()
    return unwrapping

def get_stack(cleaned_df):
    df_list = []
    country_list = cleaned_df.index.tolist()
    country_list.remove('Coverage_Range')
    for country in country_list:
        #print country
        temp_df = country_unwrap_no_index(country, cleaned_df)
        df_list.append(temp_df)
    stack_df = pd.concat(df_list).reset_index()
    stack_df.drop(columns=['index'], inplace=True)
    return stack_df

### Training of Models

*Full details can be found in the 'Training_Models' notebook.*

4 models were built for different topics predicing different indicators:

* Economy
    * GDP growth (annual %)
* Development 
    * Arable land (% of land area)
* Finance
    * Foreign direct investment, net inflows (BoP, current US dollar)
* Poverty
    * Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population)
    
There are no models built for the remaining topics, Health, Education and Environment as there's too many missing data to build an informative model.

The general workflow applied to all models built are as follow:
1. Determine an indicator to predict from the dataset built after cleaning, this is done by checking which ones have the least amount of missing data then picking an area of interest.
2. Outliers are removed by checking violin plot.
3. Missing data in the features of the dataset are filled with mean values of their respective feature.
4. Hyperparameter tuning is done on random forest regressor with gridsearch.
5. For use on the website, features importance of the model was looked into (using feature_importances_ attribute), and only the 10 most important were picked for users to use on the website.